# Semstrat transformer

The semstrat Transformer is a script designed to transform Semstrat, a budgeting file. By converting this script into a function, it can be adapted to process other budgeting files as well, offering increased flexibility and versality.

## Arguments which could be possibly passed to a function.

+ excel_file
+ sheet_name
+ rows_to_skip
+ header_rows

In [21]:
import config as c
import pandas as pd
import re

# File path and sheet name
excel_file = c.excel_file
sheet_name = c.sheet_name
rows_to_skip = 20
header_rows = 3
header_rows_list = [x for x in range(header_rows)]

# Reading excel file
df = pd.read_excel(excel_file, sheet_name=sheet_name, header=header_rows_list, skiprows=rows_to_skip)

# Keeping lvl 0 headers
lvl0_headers_to_keep = ['BASIC DATA', 'VOLUMES', 'UNIT PRICES', 'GROSS SALE', 'DISCOUNTS (incl. OCA)', 'REBATES (excl. OCA)', 'OCA', 'NNS', 'UNIT COGS', 'COGS',]

#
lvl0_headers = df.columns.get_level_values(0)

#
mask = lvl0_headers.isin(lvl0_headers_to_keep)

#
df_filtered = df.loc[:, mask]

#
df_filtered.columns = ['_'.join(map(str, col)).replace(' ', '_').replace('BASIC_DATA_vb_', '').replace('\n', '') for col in df_filtered.columns]

#
cols_to_drop = [col for col in df_filtered.columns if ('Unnamed' in col or 'Total' in col)]

df_filtered = df_filtered.drop(columns=cols_to_drop)

df_test = df_filtered.copy()

pattern = r"(\d{4})_(\w{3})"
columns_to_keep = []

for col in df_test.columns:
    if not re.search(pattern, col):
        columns_to_keep.append(col)

# Extract unique years and months from column names
years = set()
months = set()
for col in df_test.columns:
    match = re.search(pattern, col)
    if match:
        year = match.group(1)
        month = match.group(2)
        years.add(year)
        months.add(month)




In [23]:
df2 = pd.DataFrame()
for y in years:
    for m in months:
        columns_to_keep = []

        for col in df_test.columns:
            if not re.search(pattern, col):
                columns_to_keep.append(col)

        columns_to_keep = columns_to_keep + [col for col in df_test.columns if (str(y) in col and str(m) in col)]
        df_to_append = df_test[columns_to_keep]
        df_to_append['period_year'] = int(y)
        df_to_append['period_month'] = m

        column_rename_dict = {}
        for col in df_to_append.columns:
            if str(y) in col and str(m) in col:
                new_col = col.split(f"_{y}")[0]
                column_rename_dict[col] = new_col
        df_to_append.rename(columns=column_rename_dict, inplace=True)
        
        df2 = pd.concat([df2.reset_index(drop=True), df_to_append.reset_index(drop=True)])



C:\Users\floriw\AppData\Local\Temp\ipykernel_15932\530671852.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_append['period_year'] = int(y)
C:\Users\floriw\AppData\Local\Temp\ipykernel_15932\530671852.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_to_append['period_month'] = m
C:\Users\floriw\AppData\Local\Temp\ipykernel_15932\530671852.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.p

In [24]:
df2.to_excel('output.xlsx', index=False)

In [ ]:
df2.to_excel('output.xlsx', index=False)